In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import sys
import unicodedata
import re
import random
from typing import Optional

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchmetrics.classification import Accuracy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import time
import math
from tqdm import tqdm

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create Vocab class for input and output language


In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    """
    Lang(self,name:str)
    Attributes:
        word2index {dict}: Dictionary for mapping word to vocab indexes
        index2word {dict}: Dictionary for mapping vocab index to words
        word2count {count}: Dictionary for mapping word to its frequency of appearance in dataset
    Methods:
        addSentence:
            args:
                sentence {str}
            Adds words from sentence to vocab 
        addWord
    """
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

## Read languages 

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Language instances for vocabulary utilizations
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

## Filter data


In [6]:
MAX_LENGTH = 30

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p,reverse):
    if reverse is not True:
        return len(p[0].split(' ')) < MAX_LENGTH and \
            len(p[1].split(' ')) < MAX_LENGTH and \
            p[0].startswith(eng_prefixes)
    else:
        return len(p[0].split(' ')) < MAX_LENGTH and \
            len(p[1].split(' ')) < MAX_LENGTH and \
            p[1].startswith(eng_prefixes)

def filterPairs(pairs,reverse):
    return [pair for pair in pairs if filterPair(pair,reverse)]

In [7]:
import numpy as np
np.array([1,2,3]).tolist()

[1, 2, 3]

## Read and preprocess all text from txt -> Create pairs from entire data and add words from data to vocab. 

In [8]:




def prepareData(lang1, lang2, reverse=False,test_size=None):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs,reverse)
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True,1000)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Counting words...
Counted words:
fra 5173
eng 3388
['elle est toujours en mouvement', 'she s always on the go']


## Tokenize data, create tensors of the data and create dataset and data loader


In [9]:
MAX_LENGTH=30
########################################################################################################
  
def indexesFromSentence(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang,sentence):
    indexes=indexesFromSentence(lang,sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long,device=device).view(1,-1)

def tensorFromPair(pair):
    input_tensor=tensorFromSentence(input_lang,pair[0])
    output_tensor=tensorFromSentence(output_lang,pair[1])
    return(input_tensor,output_tensor)


####################################################################################################################################################


def get_dataloader(lang1,lang2,batch_size,test_ratio=0.1):
    input_lang,output_lang,pairs=prepareData(lang1,lang2,True)
    N=len(pairs)
    input_ids=torch.zeros(size=(N,MAX_LENGTH),dtype=torch.long)
    output_ids=torch.zeros(size=(N,MAX_LENGTH),dtype=torch.long)

    for idx,(inp,trg) in enumerate(pairs):
        try:
            input_tensor,output_tensor=tensorFromPair((inp,trg))
        except KeyError:
            print(f"error at {idx}th pair")
            print(inp,trg)
            continue
        input_ids[idx,:input_tensor.shape[1]]=input_tensor
        output_ids[idx,:output_tensor.shape[1]]=output_tensor
    
    test_size=int(N*test_ratio)
        
    test_idx=np.random.randint(low=0,high=N,size=test_size)
    train_idx=np.setdiff1d(np.arange(N),test_idx)
    
    train_inp,train_outp=input_ids[train_idx],output_ids[train_idx]
    test_inp,test_outp=input_ids[test_idx],output_ids[test_idx]
    
    train_data=TensorDataset(torch.LongTensor(train_inp).to(device),
                             torch.LongTensor(train_outp).to(device)
                             )
    test_data=TensorDataset(torch.LongTensor(test_inp).to(device),
                             torch.LongTensor(test_outp).to(device)
                             )
    
    train_sampler=RandomSampler(train_data)
    train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

    test_sampler=RandomSampler(test_data)
    test_dataloader=DataLoader(test_data,sampler=test_sampler,batch_size=batch_size)
    
    return input_lang,output_lang,train_dataloader,test_dataloader



## MODEL BUILDING 

### Vanilla

In [10]:
class SelfAttention(nn.Module):
    def __init__(self,input_dim,heads):
        super().__init__()
        head_dim=input_dim//heads
        self.heads=heads
        self.head_dim=head_dim
        assert input_dim%heads==0

        self.query=nn.Linear(head_dim,head_dim)
        self.key=nn.Linear(head_dim,head_dim)
        self.value=nn.Linear(head_dim,head_dim)
        self.fc_out=nn.Linear(input_dim,input_dim)
    
    def forward(self,queries,keys,values):
        N=queries.shape[0]
        ql,kl,vl=queries.shape[1],keys.shape[1],values.shape[1]   #Q,K,V
        queries=torch.transpose(queries.reshape(N,ql,self.heads,self.head_dim),1,2) #NHQD
        keys=torch.transpose(keys.reshape(N,kl,self.heads,self.head_dim),1,2)   #NHKD
        values=torch.transpose(keys.reshape(N,vl,self.heads,self.head_dim),1,2)   #NHVD   
        queries_=self.query(queries)
        keys_=self.key(keys)
        values_=self.value(values)

        attention=torch.bmm(queries_,torch.transpose(keys_-1,-2))           #NHQD . NHDK -> NHQK
        
        attention_norm=torch.softmax(attention/(self.head_dim**0.5),axis=3)
        attention_out=torch.transpose(torch.bmm(attention_norm,values_),1,2)    #NHQK . NHVD -> NHQD (V=K)
        out=self.fc_out(attention_out.reshape(N,ql,self.heads*self.head_dim))
        return out 
        

# class TransformerBlock(nn.Module):
#     def __init__(self,input_dim,heads,expansion,dropout):
#         self.expansion=input_dim*expansion
#         self.attention=SelfAttention(input_dim,heads)
#         self.dropout=nn.Dropout(dropout)
#         self.norm1=nn.LayerNorm(input_dim)
#         self.norm2=nn.LayerNorm(input_dim)
#         self.expansion_layer=nn.Sequential(
#             nn.Linear(input_dim,self.expansion),
#             nn.ReLU(),
#             nn.Linear(expansion,input_dim)
#         )   

#     def forward(self,query,key,value,mask):
#         attention=self.attention(query,key,value)



class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,dropout_p=0.1,vocab_size=None):
        super(EncoderRNN,self).__init__()
        self.hidden_size=hidden_size
        self.embedding=nn.Embedding(vocab_size,hidden_size)
        self.gru=nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.dropout=nn.Dropout(dropout_p)
        
    def forward(self,input):
        embedded=self.dropout(self.embedding(input))
        out,hidden_state=self.gru(embedded)
        return out,hidden_state


class DecoderRNN(nn.Module):
    def __init__(self,output_size,hidden_size,vocab_size=None):
        super(DecoderRNN,self).__init__()
        self.embedding=nn.Embedding(vocab_size,hidden_size)
        self.gru=nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.out=nn.Linear(hidden_size,output_size)
    
    def forward_step(self,input,hidden):
        embedded=self.embedding(input)
        output,hidden=self.gru(embedded,hidden)
        output=self.out(output)
        return output,hidden 

    def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
        N=encoder_outputs.size(0)
        decoder_input=torch.empty(N,1,dtype=torch.long,device=device).fill_(SOS_token)
        decoder_hidden=encoder_hidden
        decoder_outputs=[]
        for i in range(MAX_LENGTH):
            gru_out,decoder_hidden=self.forward_step(decoder_input,decoder_hidden)
            decoder_outputs.append(gru_out)
            if target_tensor is not None:
                decoder_input=target_tensor[:,i].unsqueeze(1)

            else:
                _,topi=gru_out.topk(1)
                decoder_input=topi.squeeze(-1).detach()

        
        decoder_outputs=torch.cat(decoder_outputs,dim=1)
        decoder_outputs=F.log_softmax(decoder_outputs,dim=-1)
        return decoder_outputs,decoder_hidden,None
    

    

        
                



        
        


### Bahdanau Attention Implementation

In [44]:
class EncoderBiRNN(nn.Module):
    def __init__(self,vocab_size,num_layers,input_size,hidden_size):
        super(EncoderBiRNN,self).__init__()
        self.embedding=nn.Embedding(vocab_size,input_size)
        self.BiRNN=nn.GRU(input_size,hidden_size, num_layers=num_layers,bidirectional=True,batch_first=True)
        
    def forward(self,x,hidden=None):
        N,max_L=x.shape
        hiddens=[]
        outputs=[]
        for i in range(max_L):
            embedding=self.embedding(x[:,i].unsqueeze(1))
            output,hidden=self.BiRNN(embedding,hidden)
            outputs.append(output)
            hiddens.append(hidden.unsqueeze(0))
        outputs=torch.cat(outputs,dim=1)            # outputs [max_L,N,1,enc_hidden_size]
        hiddens=torch.cat(hiddens,dim=0)            # hiddens [max_L,enc_nlayers*directions,N,enc_hidden_size]

        return outputs,hiddens
    
    #encoding_dict={directions=enc_directions,
        # enc_hidden_size=enc_hidden_size,
        # dec_hidden_size=enc_hidden_size,
        # enc_num_layers=enc_num_layers,
        # dec_num_layers=dec_num_layers}

        


class BahdanauAttention(nn.Module):
    def __init__(self,encoding_dict):
        super(BahdanauAttention,self).__init__()
        self.encoding_dict=encoding_dict
        self.Wa=nn.Linear(self.encoding_dict['dec_hidden_size']*self.encoding_dict['dec_num_layers'],self.encoding_dict['dec_hidden_size'])
        self.Ua=nn.Linear(self.encoding_dict["enc_hidden_size"],self.encoding_dict['dec_hidden_size'])
        self.Va=nn.Linear(self.encoding_dict["dec_hidden_size"],1)

    def forward(self,query,keys): 
        #query [dec_num_layers,N,dec_hidden]; keys  [max_L,(enc_nlayers*directions),N,enc_hidden_size]  
        #  [dec_num_layers,N,dec_hidden]-> [N,dec_num_layers,dec_hidden]-> [N,1,dec_num_layers*dec_hidden]
        query=query.transpose(0,1)
        query=query.reshape(query.size(0),1,query.size(1)*query.size(2))    
        #  [max_L,(enc_nlayers*directions),N,enc_hidden_size] -> [max_L*(enc_nlayers*directions),N,enc_hidden_size] -> [N,max_L*(enc_nlayers*directions),enc_hidden_size]
        keys=keys.reshape(-1,keys.size(2),keys.size(3)).transpose(0,1)
        #  query: [N, 1, dec_num_layers*dec_hidden] @ [dec_hidden*dec_num_layers, dec_hidden] -> [N, 1, dec_hidden]
        #  keys:  [N, max_L*(enc_nlayers*directions), enc_hidden_size] @ [enc_hidden_size, dec_hidden] -> [N, max_L*(enc_nlayers*directions), dec_hidden]
        
        #  score(addition): [N, max_L*(enc_nlayers*directions), dec_hidden] + [N, 1, dec_hidden](broadcasted) -> [N, max_L*(enc_nlayers*directions), dec_hidden]
        #  score(reduce_sum ): [N, max_L*(enc_nlayers*directions), dec_hidden] @ [dec_hidden, 1] -> [N, max_L*(enc_nlayers*directions), 1]                
        scores=self.Va(torch.tanh(self.Wa(query)+self.Ua(keys)))
        #  score: [N, max_L*(enc_nlayers*directions), 1] -> [N, max_L*(enc_nlayers*directions)] -> [N, 1, max_L*(enc_nlayers*directions)]  
        scores=scores.squeeze().unsqueeze(1)
        
        #softmax(weights)
        weights=F.softmax(scores,dim=-1)
        #  [N,1,max_L*(enc_nlayers*directions)] @ [N,max_L*(enc_nlayers*directions),enc_hidden_size] -> [N,1,enc_hidden_size]
        context=torch.bmm(weights,keys) 

        return context,weights

class BahdanauDecoder(nn.Module):
    def __init__(self,vocab_size,hidden_size,num_layers,encoding_dict):
        super(BahdanauDecoder,self).__init__()
        self.encoding_dict=encoding_dict
        self.encoding_dict['dec_hidden_size']=hidden_size                
        self.encoding_dict['dec_num_layers']=num_layers
        self.embedding=nn.Embedding(vocab_size,hidden_size)                            # Embedding Layer: [ vocab_size, enc_hidden_dims ]
        self.gru=nn.GRU(
            input_size=self.encoding_dict['enc_hidden_size']+self.encoding_dict['dec_hidden_size'],       #Inputs: [context+input_concat,decoder_hidden_state]
            hidden_size=self.encoding_dict['dec_hidden_size'],                                       #Outputs: [lstm_final_layer_activations,dec_hidden_state]
            num_layers=self.encoding_dict['dec_num_layers'],
            batch_first=True,
            bidirectional=encoding_dict['bidirectional']
            )
        if self.gru.bidirectional:
            self.encoding_dict['dec_directions']=2
            self.encoding_dict['dec_num_layers']*=self.encoding_dict['dec_directions']
        else:
            self.encoding_dict['dec_directions']=1                                    # In the case our decoder is bidirectional we would need to change the 
            
        self.attention=BahdanauAttention(self.encoding_dict)                           # Inputs: [t-1_dec_hidden_state,all_enc_hidden_states] | Outputs: [context_vector,attention_weights]
        self.fcout=nn.Linear(self.encoding_dict['dec_hidden_size']*self.encoding_dict['dec_directions'],vocab_size)

    def forward(self,encoder_hiddens,target_tensor=None):
        MAX_LENGTH,N=encoder_hiddens.size(0),encoder_hiddens.size(2)
        decoder_input=torch.empty((N,1),dtype=torch.long).fill_(SOS_token).to(device)                   #create empty input
        decoder_outputs=[]  #output token cache
        attention_weights=[]  #atttention weights cache
        decoder_hidden=torch.zeros((self.encoding_dict['dec_num_layers'],N,self.encoding_dict['dec_hidden_size'])).to(device)
        for i in range(MAX_LENGTH):
            decoder_output,decoder_hidden,attn_weight=self.forward_step(decoder_input,encoder_hiddens,decoder_hidden)
            if target_tensor!=None:
                decoder_input=target_tensor[:,i].unsqueeze(1)               #Teacher Forcing with groundtruth label inputs
            else:
                _,topi=decoder_output.topk(1)                               #Predicted output to input
                decoder_input=topi.squeeze(-1).detach()
            
            
            
            decoder_outputs.append(decoder_output)
            attention_weights.append(attn_weight)
        decoder_outputs=F.log_softmax(torch.cat(decoder_outputs,dim=1),dim=-1)
        attention_weights=torch.cat(attention_weights,dim=1)
        return decoder_outputs,attention_weights
    
    def forward_step(self,input,encoder_states,decoder_hidden):
        embedded=self.embedding(input)          #[batch_size,1] -> [batch_size,1,decoder_hidden]
        context,weights=self.attention(query=decoder_hidden,keys=encoder_states) #[[dec_num_layers,N,dec_hidden];[max_L,(enc_nlayers*directions),N,enc_hidden_size]] ->[N,1,enc_hidden_size];[N,1,max_L]    
        nn_inp=torch.cat([context,embedded],dim=-1)                         # [N,1,enc_hidden_size+dec_hidden]
        decoder_output,decoder_hidden=self.gru(nn_inp,decoder_hidden)       # [[N,1,enc_hidden_size+dec_hidden], [dec_num_layers,N,dec_hidden]] -> [N,1,dec_hidden_state],[dec_num_layers,N,dec_hidden]                                              
        decoder_output=self.fcout(decoder_output)                           # [N,1,dec_hidden_state] -> [N,1,output_vocab_size]
        return decoder_output,decoder_hidden,weights                

### Luong Attention Implementation

In [39]:
class EncoderLSTM(nn.Module):
    def __init__(self,vocab_size,input_size,hidden_size,num_layers,encoding_dict):
        super(EncoderLSTM,self).__init__()
        self.encoding_dict=encoding_dict
        self.embedding=nn.Embedding(vocab_size,input_size)
        self.lstm=nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,batch_first=True,bidirectional=self.encoding_dict['enc_bidirection'])
         
    def forward(self,x,state_cell=None):
        hiddens=[]
        N,max_L=x.shape
        for i in range(max_L):
            embedded=self.embedding(x[:,i].unsqueeze(1))
            if not state_cell:
                _,state_cell=self.lstm(embedded)
            else:
                _,state_cell=self.lstm(embedded,state_cell)
            hidden=state_cell[0]
            hiddens.append(hidden.unsqueeze(2))
        hiddens=torch.cat(hiddens,dim=2)

        #encoder_hidden_state (last layer) [directions,N, MAX_L, encoder_hidden_dims](we use only the last hidden layers of both encoder and decoder hidden_states)
        #[directions,N, MAX_L, encoder_hidden_dims] -> [N, MAX_L,directions,encoder_hidden_dims] ->[N, MAX_L,directions*encoder_hidden_dims]
        final_layer_hiddens=hiddens[-1*self.encoding_dict['enc_directions']:].permute(1,2,0,3)
        final_layer_hiddens=final_layer_hiddens.reshape(final_layer_hiddens.size(0),final_layer_hiddens.size(1),final_layer_hiddens.size(2)*final_layer_hiddens.size(3))
        return None, final_layer_hiddens                  #[N, MAX_L,directions*encoder_hidden_dims]

# encoding_dict={
#     'enc_hidden_size':enc_hidden_size,
#     'enc_num_layers': enc_num_layers,
#     'dec_hidden_size':dec_hidden_size,
#     "dec_num_layers":dec_num_layers     
# }

########################################################### 
#eH: eH*directions; 
#H: H*directions; 
#dH: dH*directions;
#hidden_dims: hidden_size*directions; 
###########################################################

class dotAttention(nn.Module):
    def __init__(self,encoding_dict):                   #added _ for consistency with rest of the multiplicative techniques
        super(dotAttention,self).__init__()
        assert encoding_dict['enc_hidden_size']*encoding_dict['enc_directions']==encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'], "Hidden dims need to be same for both encoder states and decoder states for dotAttention"
    def forward(self,dec_hidden_state,enc_hidden_states):                               
        dot=torch.bmm(enc_hidden_states,dec_hidden_state.transpose(-1,-2)).transpose(-1,-2)
        # dot=torch.einsum("N1H,NLH->N1L",[dec_hidden_state,enc_hidden_states])
        attn_w=F.softmax(dot,dim=-1)        # NIL @
        context=torch.bmm(attn_w,enc_hidden_states)
        return context,attn_w                       #context: N,L,eH
        
class generalDotAttention(nn.Module):
    def __init__(self,encoding_dict):
        super(generalDotAttention,self).__init__()
        self.W_a=nn.Linear(encoding_dict['enc_hidden_size']*encoding_dict['enc_directions'],encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'])
    def forward(self,dec_hidden_state,enc_hidden_states):
        enc_hidden_alligned=self.W_a(enc_hidden_states)
        dec_hidden_state=dec_hidden_state.transpose(-1,-2)                    ## N,1,H -> N,H,1
        dot=torch.bmm(enc_hidden_alligned,dec_hidden_state).transpose(-1,-2)  ## N,L,H @ N,H,1 -> N,L,1 -> N,1,L
        attn_w=F.softmax(dot,dim=-1)        # N,1,L
        context=torch.bmm(attn_w,enc_hidden_states)  ## N,1,L @ N,L,eH
        return context,attn_w                               #context: N,1,eH       
        
class concatAttention(nn.Module):
    def __init__(self,encoding_dict):
        super(concatAttention,self).__init__()
        self.W_a=nn.Linear(encoding_dict['enc_hidden_size']*encoding_dict['enc_directions']+encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'],encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'])
        self.v_a=nn.Linear(encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'],1)
    def forward(self,dec_hidden_state,enc_hidden_states):
        concat_=torch.concat((dec_hidden_state.expand(dec_hidden_state.size(0),enc_hidden_states.size(1),dec_hidden_state.size(2)),enc_hidden_states),dim=-1)      #N,1,dH -> N,L,dH c N,L,eH -> N,L,dH+eH  (dH==H)
        merged=torch.tanh(self.W_a(concat_))                #N,L,dH+eH -> N,L,H        
        attn_w=F.softmax(self.v_a(merged).transpose(-1,-2),dim=-1)  #N,L,H -> softmax(N,1,L)
        context=torch.bmm(attn_w,enc_hidden_states)     ## N,1,L @ N,L,eH
        return context,attn_w                     #context: N,1,eH ; attn_w:N,1,L

class LuongAttention(nn.Module):
    def __init__(self,encoding_dict:dict,luong_variant:str='general',luong_type:Optional[str]='global'):
        super(LuongAttention,self).__init__()
        self.attn_={'concat':concatAttention,'general':generalDotAttention,'dot':dotAttention}
        self.attention=self.attn_[luong_variant](encoding_dict)
        self.type_=luong_type
        if self.type_=='local':
            max_L,d=encoding_dict['max_L'],encoding_dict['luong_d']
            if d>max_L:
                raise IndexError("Sorry, but local span cannot be greater than the max sequence length")
            self.max_L=max_L
            self.d=d
            self.W_p=nn.Linear(encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'],encoding_dict['max_L'])
            self.V_p=nn.Linear(encoding_dict['max_L'],1)

    def local_p(self,decoder_hidden_state):
        dec_sum=torch.tanh(self.W_p(decoder_hidden_state))
        L_ratio=torch.sigmoid(self.V_p(dec_sum))
        return self.max_L*L_ratio.squeeze().type(torch.int64)

    def enc_hidden_localize(self,enc_hidden_states,p):
        indexed=[]
        max_l=enc_hidden_states.size(1)
        assert (self.d>0 and self.d<max_l), "please make sure window size is greater than zero and less than max sequence length"
        for i,idx in zip(enc_hidden_states,p):
            low=idx-self.d if idx-self.d>=0 else 0
            high=idx+self.d if idx+self.d<=max_l else max_l
            indexed.append(i[low:high,:].unsqueeze(0))
        return torch.cat(indexed,dim=0)

    def forward(self,dec_hidden_state,enc_hidden_states):
        if self.type_=='local':
            p=self.local_p(dec_hidden_state)
            enc_hidden_states=self.enc_hidden_localize(enc_hidden_states,p)           #[N,max_L,hidden_dims] -> [N,p-d:p+d,hidden_dims]
        context,attn_weights=self.attention(dec_hidden_state,enc_hidden_states)
        return context,attn_weights

class LuongDecoder(nn.Module):
    def __init__(self,vocab_size,hidden_size,num_layers,attn_variant,attn_type,encoding_dict):
        super(LuongDecoder,self).__init__()
        self.encoding_dict=encoding_dict
        self.embedding=nn.Embedding(vocab_size,hidden_size)
        self.lstm=nn.LSTM(input_size=hidden_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True,
                          bidirectional=encoding_dict['dec_bidirection']
                          )
        
        self.attention=LuongAttention(encoding_dict,luong_variant=attn_variant,luong_type=attn_type)
        self.W_c=nn.Linear(encoding_dict["enc_hidden_size"]*encoding_dict["enc_directions"]+encoding_dict["dec_hidden_size"],hidden_size)
        self.fcout=nn.Linear(hidden_size,vocab_size)
        
    def forward(self,enc_hidden_states,target_tensor=None):
        N,max_L=enc_hidden_states.shape[:2]
        decoder_input=torch.empty((N,1),dtype=torch.long,device=device).fill_(SOS_token)
        dec_hidden_cell=None
        dec_outputs,attn_weights=[],[]
        for i in range(max_L):
            decoder_output,dec_hidden_cell,attn_w=self.forward_step(decoder_input,enc_hidden_states,dec_hidden_cell)
            if target_tensor==None:
                _,topi=decoder_output.topk(1)                               # N,1,vocab_size -> N,1,1                      
                decoder_input=topi.squeeze(-1).detach()                     # N,1,1 -> N,1 
            else:
                decoder_input=target_tensor[:,i].unsqueeze(1)               # N -> N,1
            dec_outputs.append(decoder_output)
            attn_weights.append(attn_w)
        dec_outputs=F.log_softmax(torch.cat(dec_outputs,dim=1),dim=-1)
        attn_weights=torch.cat(attn_weights,dim=1)
        return dec_outputs,attn_weights            


    def forward_step(self,input,enc_hidden_states,dec_hidden_cell=None):
        embedded=self.embedding(input)
        if not dec_hidden_cell:
            _,dec_hidden_cell=self.lstm(embedded)
        else:
            _,dec_hidden_cell=self.lstm(embedded,dec_hidden_cell)
        dec_hidden_state=dec_hidden_cell[0][-1*self.encoding_dict['dec_directions']:]
        dec_hidden_state=dec_hidden_state.transpose(0,1)
        dec_hidden_state=dec_hidden_state.reshape(dec_hidden_state.size(0),dec_hidden_state.size(1)*dec_hidden_state.size(2)).unsqueeze(1)      #(hidden,cell) -> hidden: directions*Nlayers,N,dH -> directions,N,dH -> N,dH*directions ->  N,1,dH*directions 
        context,attn_w=self.attention(dec_hidden_state,enc_hidden_states)
        context.shape
        context_cat=torch.cat([context,embedded],dim=-1)            # N,1,dH ; N,1,eH -> N,1,dH+eH
        decoder_output=torch.tanh(self.W_c(context_cat))            # N,1,dH+eH -> N,1,dH
        # decoder_output,decoder_hidden_cell=self.lstm(decoder_output,dec_hidden_cell)
        decoder_output=self.fcout(decoder_output)                   # N,1,dH -> N,1,vocab_size
        return decoder_output,dec_hidden_cell,attn_w

        

            

        

In [13]:

encoding_dict['max_L']=MAX_LENGTH
W_p=nn.Linear(encoding_dict['dec_hidden_size']*encoding_dict['dec_directions'],encoding_dict['max_L'])
V_p=nn.Linear(encoding_dict['max_L'],1)


def local_p(decoder_hidden_state):
    dec_sum=torch.tanh(W_p(decoder_hidden_state))
    print(dec_sum.shape,V_p.weight.shape)
    L_ratio=torch.sigmoid(V_p(dec_sum))
    return MAX_LENGTH*L_ratio.squeeze()

In [332]:
p=local_p(torch.rand(35,1,encoding_dict['dec_directions']*encoding_dict['dec_hidden_size'])).type(torch.int64)
tensor=torch.rand(35,50,256)
enc_hidden_encased(tensor,p,3).shape

torch.Size([35, 1, 30]) torch.Size([1, 30])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])


torch.Size([35, 6, 256])

In [320]:
def enc_hidden_encased(enc_hidden_states,p,d):
    indexed=[]
    max_l=enc_hidden_states.size(1)
    for i,idx in zip(enc_hidden_states,p):
        low=idx-d if idx-d>=0 else 0
        high=idx+d if idx+d<=max_l else max_l
        indexed.append(i[low:high,:].unsqueeze(0))
        print(i[idx-d:idx+d,:].shape)
    return torch.cat(indexed,dim=0)

In [300]:
torch.rand(5,30,256).gather(dim=1,index=p)

RuntimeError: Index tensor must have the same number of dimensions as input tensor

## Training and Plotting Utils

In [26]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points,range(len(points)))
    plt.show()


def eval_train(encoder,decoder,data_loader,accuracy):
    encoder.eval(),decoder.eval()
    accuracy_=0
    n_eval=0
    with torch.no_grad():
        for idx,(input_,targets) in enumerate(data_loader):
            _,encoder_hidden=encoder(input_)
            preds,attn_weights=decoder(encoder_hidden)
            for pred,target in zip(preds,targets):
                accuracy_+=accuracy(pred,target)
                n_eval+=1
    encoder.train(),decoder.train()
    return accuracy_.item()/n_eval


def train_bahdanau_luong_epoch(encoder,decoder,dataloader,criterion,encoder_optimizer,decoder_optimizer):
    total_loss=0
    for data in tqdm(dataloader):
        x,y=data
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        _,encoder_hidden=encoder(x)
        decoder_outputs,attn_weights=decoder(encoder_hidden,target_tensor=y)
        loss=criterion(decoder_outputs.reshape(-1,decoder_outputs.size(-1)),y.view(-1))
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss+=loss.item()
    return total_loss / len(dataloader)

def train_bahdanau_luong(epochs,encoder,decoder,train_dataloader,test_dataloader,lr,print_every,plot_every):
    plot_losses=[]
    print_losses=[]
    plot_loss_total=0
    print_loss_total=0
    decoder_optimizer=torch.optim.Adam(params=decoder.parameters(),lr=lr)
    encoder_optimizer=torch.optim.Adam(params=encoder.parameters(),lr=lr)
    criterion=nn.NLLLoss()
    accuracy=Accuracy(task="multiclass",num_classes=decoder.fcout.out_features,top_k=2).to(device)
    # pbar=tqdm(total=epochs,position=0,desc='Training Progress')
    # loss_log=tqdm(position=1,total=0,bar_foramt='{desc}')
    for i in range(1,epochs+1):
        avg_loss=train_bahdanau_luong_epoch(encoder,decoder,train_dataloader,criterion,encoder_optimizer,decoder_optimizer)
        val_acc=eval_train(encoder,decoder,test_dataloader,accuracy)
        plot_loss_total+=avg_loss
        print_loss_total+=avg_loss
        if i%print_every==0:
            print_losses.append(print_loss_total/print_every)
            print(f"Epoch {i} / {epochs} :  Loss {print_loss_total/print_every}   |   Validation Accuracy {val_acc}")
            print_loss_total=0
        # loss_log.set_description_str(f"Epoch {i} / {epochs} :  Loss {print_loss_total/print_every}")
        # pbar.update(1)
        if i%plot_every==0:
            plot_losses.append(plot_loss_total/plot_every)
            plot_loss_total=0
    showPlot(plot_losses)
    

## Testing 

In [40]:
enc_hidden_size=512
enc_num_layers=3
encoding_dict={'enc_hidden_size':enc_hidden_size,
               "enc_num_layers":enc_num_layers,
               'enc_bidirection':True,
               'dec_bidirection':False,
               'dec_hidden_size':512,
               'max_L':30,
               'luong_d':3}
encoding_dict['enc_directions']=1 if not encoding_dict['enc_bidirection'] else 2
encoding_dict['dec_directions']=1 if not encoding_dict['dec_bidirection'] else 2
# input_lang,output_lang,train_data,test_data=get_dataloader('eng','fra',25)
enlstm=EncoderLSTM(vocab_size=input_lang.n_words,input_size=256,hidden_size=enc_hidden_size,num_layers=enc_num_layers,encoding_dict=encoding_dict).to(device)
dclstm=LuongDecoder(vocab_size=output_lang.n_words,hidden_size=512,num_layers=1,attn_variant='concat',attn_type='global',encoding_dict=encoding_dict).to(device)

In [37]:
encoding_dict

{'enc_hidden_size': 512,
 'enc_num_layers': 3,
 'enc_bidirection': True,
 'dec_bidirection': False,
 'dec_hidden_size': 512,
 'max_L': 30,
 'luong_d': 3,
 'enc_directions': 2,
 'dec_directions': 1}

In [24]:
encoder_hidden_states=enlstm(x)[1]
dclstm(encoder_hidden_states,y)[0].shape

torch.Size([25, 30, 3388])

In [41]:
train_bahdanau_luong(epochs=10,encoder=enlstm,decoder=dclstm,train_dataloader=train_data,test_dataloader=test_data,lr=0.002,print_every=1,plot_every=1)

100%|██████████| 474/474 [01:09<00:00,  6.85it/s]


Epoch 1 / 10 :  Loss 0.7774928457374815   |   Validation Accuracy 0.8473097458994836


100%|██████████| 474/474 [01:09<00:00,  6.83it/s]


Epoch 2 / 10 :  Loss 0.471208248397469   |   Validation Accuracy 0.8939839416363811


100%|██████████| 474/474 [01:10<00:00,  6.77it/s]


...

100%|██████████| 474/474 [01:03<00:00,  7.49it/s]


Epoch 9 / 10 :  Loss 0.06484233831224437   |   Validation Accuracy 0.9253780537191565


100%|██████████| 474/474 [01:03<00:00,  7.43it/s]


Epoch 10 / 10 :  Loss 0.06254603692137868   |   Validation Accuracy 0.9321883069529456


/tmp/ipykernel_229231/3006362193.py:8: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [45]:
enc_hidden_size=512
enc_num_layers=2
device=torch.device('cuda')
encoding_dict={'enc_hidden_size':enc_hidden_size,"enc_num_layers":enc_num_layers,'bidirectional':True}
enbahdanau=EncoderBiRNN(vocab_size=input_lang.n_words,num_layers=2,input_size=256,hidden_size=512).to(device)
decbahdanau=BahdanauDecoder(vocab_size=output_lang.n_words,hidden_size=512,num_layers=2,encoding_dict=encoding_dict).to(device)

In [46]:
train_bahdanau_luong(epochs=10,encoder=enbahdanau,decoder=decbahdanau,train_dataloader=train_data,test_dataloader=test_data,lr=0.002,print_every=1,plot_every=1)

100%|██████████| 474/474 [01:13<00:00,  6.47it/s]


Epoch 1 / 10 :  Loss 0.7413054793318615   |   Validation Accuracy 0.8568235716932862


100%|██████████| 474/474 [01:10<00:00,  6.71it/s]


Epoch 2 / 10 :  Loss 0.43619052078653486   |   Validation Accuracy 0.8835526738535291


100%|██████████| 474/474 [01:10<00:00,  6.69it/s]


...

100%|██████████| 474/474 [01:09<00:00,  6.86it/s]


Epoch 9 / 10 :  Loss 0.11951883757714858   |   Validation Accuracy 0.9094890194625096


100%|██████████| 474/474 [01:09<00:00,  6.86it/s]


Epoch 10 / 10 :  Loss 0.11853694210795662   |   Validation Accuracy 0.9124228164451033


/tmp/ipykernel_229231/3006362193.py:8: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
from itertools import chain,product,combinations

In [ ]:
list(combinations("ABCD",2))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]

In [ ]:
# python program to demonstrate
# unique combination of two lists
# using zip() and permutation of itertools
 
# import itertools package
import itertools
from itertools import permutations
 
# initialize lists
list_1 = ["a", "b", "c","d"]
list_2 = [1,4,9]
unique_combinations = []
permut = itertools.permutations(list_1, len(list_2))

for comb in permut:
    zipped = zip(comb, list_2)
    unique_combinations.append(list(zipped))
print(unique_combinations)


[[('a', 1), ('b', 4), ('c', 9)], [('a', 1), ('b', 4), ('d', 9)], [('a', 1), ('c', 4), ('b', 9)], [('a', 1), ('c', 4), ('d', 9)], [('a', 1), ('d', 4), ('b', 9)], [('a', 1), ('d', 4), ('c', 9)], [('b', 1), ('a', 4), ('c', 9)], [('b', 1), ('a', 4), ('d', 9)], [('b', 1), ('c', 4), ('a', 9)], [('b', 1), ('c', 4), ('d', 9)], [('b', 1), ('d', 4), ('a', 9)], [('b', 1), ('d', 4), ('c', 9)], [('c', 1), ('a', 4), ('b', 9)], [('c', 1), ('a', 4), ('d', 9)], [('c', 1), ('b', 4), ('a', 9)], [('c', 1), ('b', 4), ('d', 9)], [('c', 1), ('d', 4), ('a', 9)], [('c', 1), ('d', 4), ('b', 9)], [('d', 1), ('a', 4), ('b', 9)], [('d', 1), ('a', 4), ('c', 9)], [('d', 1), ('b', 4), ('a', 9)], [('d', 1), ('b', 4), ('c', 9)], [('d', 1), ('c', 4), ('a', 9)], [('d', 1), ('c', 4), ('b', 9)]]


In [ ]:
from configparser import ConfigParser

config=ConfigParser()

config.read('config.ini')


encoding_dict={key:int(value) if value.isdigit() else value for key,value in config._sections['model'].items()}
encoding_dict['enc_directions']=2 if encoding_dict['enc_bidirection'].lower()=='True' else 1
encoding_dict['dec_directions']=2 if encoding_dict['dec_bidirection'].lower()=='True' else 1


batch_size=int(config.get("training","batch_size"))
test_size=float(config.get('training','test_size'))
MAX_LENGTH=int(config.get('training','max_length'))
epochs=int(config.get("training","epochs"))




if (test_size>1 or test_size<0):
    raise ValueError("test_size must be a ratio") 

if config.get("model","attention_variant").lower()=='luong':
    type_=config.get("Luong","luong_type")
    attention_variant=config.get("Luong","luong_variant")
    if type_=="local":
        d=int(config.get("Luong","d"))
        max_L_d=(MAX_LENGTH,d)
elif config.get("model","attention_variant").lower()=='bahdanau':
    pass
else: 
    raise ValueError("Attention variants can either be 'luong' or 'bahdanau'")


    

In [ ]:
config.read('config.ini')
if config.get("model","dec_bidirection").lower()=="True":


In [71]:
config._sections['model']

NameError: name 'config' is not defined

In [ ]:
config._sections['training']

{'batch_size': '', 'max_length': ''}